References:
* https://pythonhosted.org/pyglet/programming_guide/controlling_playback.html

## Set Things Up

In [1]:
## Import all the things

import pyglet
import time
import sys
import numpy as np
import pandas as pd

## Code to play sound

Popup window will open when code is executed. When you close window, sound will stop.

In [18]:
def play_background(audio_file):
    """
    Play the background song
    
    Args: 
        audio_file (str): File path of WAV audio file.
    """
    
    # Instantiate player
    # Remove any preexisting audio from queue
    
    global player
    
    try:
        player
    except NameError: player = None

    if player is None:
        player = pyglet.media.Player()
    else:
        player.next_source()
    
    ## Add new audio file to queue

    source = pyglet.media.load(audio_file, streaming=False)

    player.queue(source) 
    player.play()

In [19]:
class Conductor:
    """
    Class that ingests and sonifies data.
    """

    def __init__(self, data):
        self.data = data
    
    def background(self, background):
        """ Select genre of music to use for sonification. """
        
        self.background = background
    
    def ding(self, cols):
        """ Assign columns for DINGS. """
        
        self.ding_cols = cols
    
    def play(self):
        """ Sonify and play data. """
        
        window = pyglet.window.Window()
        play_background(self.background)
        pyglet.app.run()

Test...

In [20]:
dat = pd.read_csv('data/intrusion_detection.csv')

test = Conductor(dat)
test.background('sound-files/Yellow_16_NZD.wav')
test.play()

## Code to loop through data frame

I want to loop through each line of dataframe to:
* Decide whether to play DING
* Have a scrolling display in popup window

In [10]:
def loop_rows(df):
    """
    Loop through dataframe, row-by-row. Pause for 1.5 seconds between rows.
    """

    for i in range(df.shape[0]):
        time.sleep(1.5)
        print(str(df.iloc[i].to_frame().T))